<a href="https://colab.research.google.com/github/dinuwanfernando/10822750-final-year-project/blob/main/loan_prediction_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loan Prediction Model - Version 2**

In [55]:
#Importing libraries
import pandas as pd
import numpy as np
import math
import datetime
from datetime import datetime
import seaborn as sns
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from math import sqrt
import matplotlib.pyplot as plt
!pip install prophet
from prophet import Prophet

In [56]:
df= pd.read_csv('/content/train.csv')
print(df)

      Loan_ID  Gender Married Dependents     Education Self_Employed  \
0    LP001002    Male      No          0      Graduate            No   
1    LP001003    Male     Yes          1      Graduate            No   
2    LP001005    Male     Yes          0      Graduate           Yes   
3    LP001006    Male     Yes          0  Not Graduate            No   
4    LP001008    Male      No          0      Graduate            No   
..        ...     ...     ...        ...           ...           ...   
609  LP002978  Female      No          0      Graduate            No   
610  LP002979    Male     Yes         3+      Graduate            No   
611  LP002983    Male     Yes          1      Graduate            No   
612  LP002984    Male     Yes          2      Graduate            No   
613  LP002990  Female      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0               5849                0.0         NaN           

In [57]:
df_v1= df.drop(['Loan_ID','Gender'], axis=1)
df_v1= df_v1.dropna()
print(df_v1)

    Married Dependents     Education Self_Employed  ApplicantIncome  \
1       Yes          1      Graduate            No             4583   
2       Yes          0      Graduate           Yes             3000   
3       Yes          0  Not Graduate            No             2583   
4        No          0      Graduate            No             6000   
5       Yes          2      Graduate           Yes             5417   
..      ...        ...           ...           ...              ...   
609      No          0      Graduate            No             2900   
610     Yes         3+      Graduate            No             4106   
611     Yes          1      Graduate            No             8072   
612     Yes          2      Graduate            No             7583   
613      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1               1508.0       128.0             360.0             1.0   
2  

In [58]:
num_variations_dep= df_v1['Dependents'].unique()
print(num_variations_dep)
num_variations_edu= df_v1['Education'].unique()
print(num_variations_edu)
num_variations_self= df_v1['Self_Employed'].unique()
print(num_variations_self)
num_variations_prop= df_v1['Property_Area'].unique()
print(num_variations_prop)
num_variations_cred= df_v1['Credit_History'].unique()
print(num_variations_cred)

['1' '0' '2' '3+']
['Graduate' 'Not Graduate']
['No' 'Yes']
['Rural' 'Urban' 'Semiurban']
[1. 0.]


In [59]:
#removing the text values
def map_to_binary(value):
  return 1 if value == 'Yes' else 0

In [60]:
df_v1['Married_1']= df_v1['Married'].apply(map_to_binary)

print(df_v1)

    Married Dependents     Education Self_Employed  ApplicantIncome  \
1       Yes          1      Graduate            No             4583   
2       Yes          0      Graduate           Yes             3000   
3       Yes          0  Not Graduate            No             2583   
4        No          0      Graduate            No             6000   
5       Yes          2      Graduate           Yes             5417   
..      ...        ...           ...           ...              ...   
609      No          0      Graduate            No             2900   
610     Yes         3+      Graduate            No             4106   
611     Yes          1      Graduate            No             8072   
612     Yes          2      Graduate            No             7583   
613      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1               1508.0       128.0             360.0             1.0   
2  

In [61]:
def map_to_binary_edu(value):
  return 1 if value == 'Graduate' else 0

In [62]:
df_v1['Education_1']=df_v1['Education'].apply(map_to_binary_edu)
print(df_v1)

    Married Dependents     Education Self_Employed  ApplicantIncome  \
1       Yes          1      Graduate            No             4583   
2       Yes          0      Graduate           Yes             3000   
3       Yes          0  Not Graduate            No             2583   
4        No          0      Graduate            No             6000   
5       Yes          2      Graduate           Yes             5417   
..      ...        ...           ...           ...              ...   
609      No          0      Graduate            No             2900   
610     Yes         3+      Graduate            No             4106   
611     Yes          1      Graduate            No             8072   
612     Yes          2      Graduate            No             7583   
613      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1               1508.0       128.0             360.0             1.0   
2  

In [63]:
def map_to_binary_self(value):
  return 1 if value == 'No' else 0

In [64]:
df_v1['Self_Employed_1']=df_v1['Self_Employed'].apply(map_to_binary_self)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1


In [65]:
def map_to_binary_property(value):
  if value == 'Urban':
    return 1
  elif value == 'Semiurban':
    return 2
  else:
    return 0

In [66]:
df_v1['Property_Area_1']= df_v1['Property_Area'].apply(map_to_binary_property)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1,Property_Area_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,0
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1,1
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1,1
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1,0
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1,0
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1,1


In [67]:
df_v1['Dependents_1']=df_v1['Dependents'].replace('3+', 3)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,0,1
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,0
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1,1,0
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1,1,0
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1,0,0
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1,0,3
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1,1,2


In [68]:
def map_to_binary_loan(value):
  return 1 if value == 'Y' else 0

In [69]:
df_v1['Loan_Status_1']=df_v1['Loan_Status'].apply(map_to_binary_loan)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,Loan_Status_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,0,1,0
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,0,1
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1,1,0,1
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1,1,0,1
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1,0,0,1
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1,0,3,1
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1,1,2,1


In [70]:
df_v2= df_v1.drop(['Married','Education','Self_Employed','Property_Area','Loan_Status','Dependents'], axis=1)
df_v2['Dependents_1']=df_v2['Dependents_1'].astype(int)
df_v2

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,Loan_Status_1
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,1,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,1,0,1,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,1,0,1
5,5417,4196.0,267.0,360.0,1.0,1,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,0,1,1,0,0,1
610,4106,0.0,40.0,180.0,1.0,1,1,1,0,3,1
611,8072,240.0,253.0,360.0,1.0,1,1,1,1,1,1
612,7583,0.0,187.0,360.0,1.0,1,1,1,1,2,1


In [71]:
def map_to_binary_loan(value):
  return 1 if value == 'Y' else 0

In [72]:
df_v1['Loan_Status_1']=df_v1['Loan_Status'].apply(map_to_binary_loan)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,Loan_Status_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,0,1,0
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,0,1
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1,1,0,1
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1,1,0,1
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1,0,0,1
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1,0,3,1
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1,1,2,1


In [73]:
df_v2= df_v1.drop(['Married','Education','Self_Employed','Property_Area','Loan_Status','Dependents'], axis=1)
df_v2['Dependents_1']=df_v2['Dependents_1'].astype(int)
df_v2

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,Loan_Status_1
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,1,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,1,0,1,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,1,0,1
5,5417,4196.0,267.0,360.0,1.0,1,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,0,1,1,0,0,1
610,4106,0.0,40.0,180.0,1.0,1,1,1,0,3,1
611,8072,240.0,253.0,360.0,1.0,1,1,1,1,1,1
612,7583,0.0,187.0,360.0,1.0,1,1,1,1,2,1


In [74]:
from sklearn.model_selection import train_test_split

In [75]:
test_size = 0.2  # Proportion for the test set (20% in this case)
random_state = 42  # Ensures reproducibility (optional)

train_index, test_index = train_test_split(df_v2.index, test_size=test_size, random_state=random_state)

train_df = df_v2.loc[train_index]  # Select rows for training set based on indexes
test_df = df_v2.loc[test_index]   # Select rows for testing set based on indexes

In [76]:
print(train_df)
train_df['Dependents_1']=train_df['Dependents_1'].astype(int)
train_df.dtypes

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
475            16525             1014.0       150.0             360.0   
159             4583             5625.0       255.0             360.0   
525            17500                0.0       400.0             360.0   
522             5677             1424.0       100.0             360.0   
214             3173             3021.0       137.0             360.0   
..               ...                ...         ...               ...   
139             4200             1430.0       129.0             360.0   
341             2378                0.0        46.0             360.0   
438            10416                0.0       187.0             360.0   
546             3358                0.0        80.0              36.0   
135             4000             7750.0       290.0             360.0   

     Credit_History  Married_1  Education_1  Self_Employed_1  Property_Area_1  \
475             1.0          1            

ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Married_1              int64
Education_1            int64
Self_Employed_1        int64
Property_Area_1        int64
Dependents_1           int64
Loan_Status_1          int64
dtype: object

In [77]:
print(test_df)
test_df['Dependents_1']=test_df['Dependents_1'].astype(int)
test_df.dtypes

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
97              1977              997.0        50.0             360.0   
68              7100                0.0       125.0              60.0   
343             3173                0.0        74.0             360.0   
361             5000             3667.0       236.0             360.0   
77              1000             3022.0       110.0             360.0   
..               ...                ...         ...               ...   
402             2500            20000.0       103.0             360.0   
455             3859                0.0        96.0             360.0   
69              4300                0.0       136.0             360.0   
276             3993             3274.0       207.0             360.0   
272             2833             1857.0       126.0             360.0   

     Credit_History  Married_1  Education_1  Self_Employed_1  Property_Area_1  \
97              1.0          1            

ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Married_1              int64
Education_1            int64
Self_Employed_1        int64
Property_Area_1        int64
Dependents_1           int64
Loan_Status_1          int64
dtype: object

In [78]:
target= 'Loan_Status_1'
features=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Married_1','Education_1','Self_Employed_1','Property_Area_1','Dependents_1']

In [79]:
#data control
X_train= train_df[features]
Y_train= train_df[target]
print(X_train)
print(Y_train)

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
475            16525             1014.0       150.0             360.0   
159             4583             5625.0       255.0             360.0   
525            17500                0.0       400.0             360.0   
522             5677             1424.0       100.0             360.0   
214             3173             3021.0       137.0             360.0   
..               ...                ...         ...               ...   
139             4200             1430.0       129.0             360.0   
341             2378                0.0        46.0             360.0   
438            10416                0.0       187.0             360.0   
546             3358                0.0        80.0              36.0   
135             4000             7750.0       290.0             360.0   

     Credit_History  Married_1  Education_1  Self_Employed_1  Property_Area_1  \
475             1.0          1            

Training Using XGBoost

In [80]:
# model_credit= xgb.XGBRegressor(objective='reg:squarederror')
# param_grid= {'n_estimators':[50,100,150,200,300,400,500,600,700,800,900,1000],
#           'learning_rate':[0.01,0.05,0.1],
#             'max_depth':[3,4,5,6,7,8,9,10],
#            'base_score':[0.5,0.6,0.7,0.8,0.9,1],
#            }

# grid_search= GridSearchCV(estimator=model_credit, param_grid= param_grid, cv=3,n_jobs=-1,verbose=2)
# grid_search.fit(X_train,np.ravel(Y_train,order='C'))
# best_params= grid_search.best_params_
# print(best_params)

In [81]:
model_credit= xgb.XGBRegressor(base_score=0.5, booster='gbtree',
                           n_estimators=1000,
                           objective='reg:squarederror',
                           max_depth=5,
                           learning_rate=0.1)

Training Using Random Forest Classifier

In [82]:
# from sklearn.ensemble import RandomForestClassifier

In [83]:
# from sklearn.model_selection import RandomizedSearchCV

In [84]:
# RandomForestClassifier()

In [85]:
# rf_grid={'n_estimators':np.arange(10,1000,10),
#   'max_features':['auto','sqrt'],
#  'max_depth':[None,3,5,10,20,30],
#  'min_samples_split':[2,5,20,50,100],
#  'min_samples_leaf':[1,2,5,10]
#  }

In [86]:
# model_credit=RandomizedSearchCV(RandomForestClassifier(),
#                   param_distributions=rf_grid,
#                    cv=5,
#                    n_iter=20,
#                   verbose=True)

In [87]:
model_credit.fit(X_train,Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [88]:
X_test_df= test_df[features]
X_test_df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1
97,1977,997.0,50.0,360.0,1.0,1,1,1,2,0
68,7100,0.0,125.0,60.0,1.0,1,0,0,1,3
343,3173,0.0,74.0,360.0,1.0,1,0,1,2,3
361,5000,3667.0,236.0,360.0,1.0,1,1,1,2,2
77,1000,3022.0,110.0,360.0,1.0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...
402,2500,20000.0,103.0,360.0,1.0,0,1,1,2,0
455,3859,0.0,96.0,360.0,1.0,1,1,1,2,2
69,4300,0.0,136.0,360.0,0.0,0,1,1,2,0
276,3993,3274.0,207.0,360.0,1.0,1,1,1,2,0


In [89]:
pred_loan_status= model_credit.predict(X_test_df)
pred_loan_status

array([ 1.0429131 ,  0.96469086,  0.31183603,  0.9838656 ,  0.83638954,
        0.09076858,  1.0044605 ,  0.1058134 ,  0.8950391 ,  0.90600204,
        0.8683719 ,  0.8642427 ,  0.86873025,  0.86353564,  0.57136774,
        0.46281153, -0.01804824,  0.9417755 ,  1.0741957 ,  0.32674527,
        0.05764614,  0.57451344,  0.58147496,  1.0559744 ,  0.99218106,
        0.93972355,  0.0228571 ,  0.00291423,  0.97043806,  0.45358947,
        0.960058  ,  0.5447624 ,  1.0240945 ,  0.8973448 ,  0.8862897 ,
        0.00263934,  0.8290262 ,  0.879614  ,  0.8853024 ,  0.37502316,
        1.0038161 ,  0.6369616 ,  1.1062781 ,  0.6053418 ,  0.7269738 ,
        1.0512271 , -0.01021801, -0.25225198,  1.0565585 ,  0.81634384,
        0.9486588 ,  0.78473204,  0.17073381,  0.8700032 ,  0.3479806 ,
       -0.14485307,  0.6603261 ,  0.6584048 ,  0.848921  ,  0.8823831 ,
        0.78243613,  0.20672044, -0.10464631, -0.0157788 ,  0.5564665 ,
        1.0757761 ,  0.7853097 ,  1.0727206 ,  1.0232102 ,  0.89

In [90]:
X_test_df2= X_test_df

In [91]:
Loan_Status_2=test_df['Loan_Status_1']
Loan_Status_2

97     1
68     1
343    1
361    1
77     0
      ..
402    1
455    1
69     0
276    1
272    1
Name: Loan_Status_1, Length: 99, dtype: int64

In [92]:
X_test_df2['actual_loan_status']=Loan_Status_2

In [93]:
X_test_df2['pred_loan_status']=pred_loan_status
X_test_df2.head(50)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,actual_loan_status,pred_loan_status
97,1977,997.0,50.0,360.0,1.0,1,1,1,2,0,1,1.042913
68,7100,0.0,125.0,60.0,1.0,1,0,0,1,3,1,0.964691
343,3173,0.0,74.0,360.0,1.0,1,0,1,2,3,1,0.311836
361,5000,3667.0,236.0,360.0,1.0,1,1,1,2,2,1,0.983866
77,1000,3022.0,110.0,360.0,1.0,1,1,0,1,1,0,0.836390
229,6400,0.0,200.0,360.0,1.0,0,1,0,0,0,1,0.090769
39,3748,1668.0,110.0,360.0,1.0,0,0,1,2,0,1,1.004460
150,6277,0.0,118.0,360.0,0.0,0,1,1,0,0,0,0.105813
581,1836,33837.0,90.0,360.0,1.0,0,1,1,1,0,0,0.895039
422,1820,1719.0,100.0,360.0,1.0,1,1,1,1,0,1,0.906002


In [94]:
#adjust as you wish based on the accuracy
def map_to_binary_loan_status_pred(value):
  return 1 if value > 0.51 else 0

In [95]:
X_test_df2['pred_loan_status_1']=X_test_df2['pred_loan_status'].apply(map_to_binary_loan_status_pred)
X_test_df2

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,actual_loan_status,pred_loan_status,pred_loan_status_1
97,1977,997.0,50.0,360.0,1.0,1,1,1,2,0,1,1.042913,1
68,7100,0.0,125.0,60.0,1.0,1,0,0,1,3,1,0.964691,1
343,3173,0.0,74.0,360.0,1.0,1,0,1,2,3,1,0.311836,0
361,5000,3667.0,236.0,360.0,1.0,1,1,1,2,2,1,0.983866,1
77,1000,3022.0,110.0,360.0,1.0,1,1,0,1,1,0,0.836390,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2500,20000.0,103.0,360.0,1.0,0,1,1,2,0,1,0.639125,1
455,3859,0.0,96.0,360.0,1.0,1,1,1,2,2,1,0.905586,1
69,4300,0.0,136.0,360.0,0.0,0,1,1,2,0,0,0.546403,1
276,3993,3274.0,207.0,360.0,1.0,1,1,1,2,0,1,0.931295,1


In [96]:
from sklearn.metrics import accuracy_score

In [97]:
actual_values=X_test_df2['actual_loan_status']
predicted_values= X_test_df2['pred_loan_status_1']

In [98]:
accuracy = accuracy_score(actual_values, predicted_values)
accuracy*100

74.74747474747475

In [99]:
import joblib

In [100]:
joblib.dump(model_credit,'loan_status_predict')

['loan_status_predict']

In [101]:
model = joblib.load('loan_status_predict')

In [102]:
data = {
    'ApplicantIncome': [3565],
    'CoapplicantIncome': [0.0],
    'LoanAmount': [120],
    'Loan_Amount_Term': [360],
    'Credit_History': [1],
    'Married_1': [0],
    'Education_1': [1],
    'Self_Employed_1': [0],
    'Property_Area_1': [1],
    'Dependents_1': [2]
}

In [103]:
df_final = pd.DataFrame(data)

In [104]:
df_final

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1
0,3565,0.0,120,360,1,0,1,0,1,2


In [105]:
model.predict(df_final)

array([0.56728464], dtype=float32)

In [106]:
result = model.predict(df_final)

In [107]:
if result > 0.51:
    print("Loan Status : Approved")
    print("Credit Score : ", int(result[0]*1000))
else:
    print("Loan Status : Not Approved")
    print("Credit Score : ", int(result[0]*1000))

Loan Status : Approved
Credit Score :  567
